In [ ]:
# 1. Mostrar algunas realidades de video processing, y no de ML. Correr sin modelo
# sobre videos de distintos fps, longitud, tamaño y calidad

# - como varía el tiempo segun los target-fps
# - como varía el tiempo segun el tamaño del frame
# - como varía el tiempo segun el codec (h264, mp4v)
# - como varía el tiempo segun la calidad del target

# 2. Mostrar cuanto tiempo extra agrega llamar al modelo

# 3. Mostrar cuanto agregan los multitrackers
# - hacer un heatmap por cada multitracker que muestre target-fps, secs-refetch,
# y duracion del proceso, usando de verde que se haya procesado en menos que lo que 
# dura el video

# chusmear https://github.com/sharkdp/hyperfine/blob/master/scripts/plot_parametrized.py

In [ ]:
import subprocess
import os
import cv2

# TODO: agregar landscape de 45sec@60fps
TEST_FILES = ['../test-videos/face-1000fps.mp4', '../test-videos/face.mp4', '../test-videos/portrait.mp4']

# sudo apt install hyperfine
HYPERFINE = ['hyperfine', '--style', 'basic', '--max-runs', '3',]

def basename(file):
    return os.path.basename(file)

def csv(file): 
    return f'benchmarks-{basename(file).split(".")[0]}.csv'

def info(file):
    video = cv2.VideoCapture(file)
    fps = video.get(cv2.CAP_PROP_FPS)
    numberframes = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video.release()
    return {'duration': numberframes / fps, 'fps': fps}


for file in TEST_FILES:
    # TODO: Armar la lista de fps en base a los fps del video, para no llamar a
    # hyperfine mas de lo normal
    subprocess.run([
        *HYPERFINE,
        '--export-csv', csv(file),
        '--parameter-list', 'targetfps', '5,15,24,30,60,90',
        f'python3 ../src/main.py {file} --target-fps {{targetfps}} --secs-refetch 0'
    ])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, len(TEST_FILES), figsize=(15, 5))
axes = axes.flatten()

for i, file in enumerate(TEST_FILES):
    videoinfo = info(file)
    df = pd.read_csv(csv(file))
    plt.sca(axes[i])
    
    plt.plot(df['parameter_targetfps'], df['mean'], marker='o', linestyle='-')
    plt.xlabel('target fps')
    plt.ylabel('Mean (seconds)')
    plt.axhline(y=videoinfo['duration'], color='red', linestyle='--', label="Video duration")
    plt.title(f"{basename(file)} ({round(videoinfo['fps'], 1)} fps)")
    plt.legend()

plt.tight_layout()
plt.show()